<h1>Allocation Service - Notebook 2</h1>
This notebook demonstrates a real world use case of the Allocation Service. We will create two orders for the same instrument for separate portfolios, along with the supporting entities. We will then simulate an EOD process where a uer books transactions from the suggested Allocations. We will then see how the Allocation Service handles executions for the next business day.

<h2> Imports </h2>
Import the required packages for this example.

In [1]:
import lusid
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
)
from helpers import AllocationServiceHelpers as helpers

import pandas as pd
import os

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.11172.0


<h2> Setup Constants </h2>
We set up everything in the same scope for simplicity.

In [2]:
scope = "AllocationService_Example2"

<h2> Upsert Instrument </h2>
We use a simple AMZN equity for this example.

In [3]:
instr_df = pd.read_csv("data/2/instruments.csv")
display(instr_df)

instrument_name client_internal currency          isin          figi  \
0  Amazon_Nasdaq_AMZN    imd_34634534      USD  US0231351067  BBG000BVPXP1   

  ticker  
0   AMZN

In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0        1       0       0

<h2> Upsert Portfolios </h2>
We have two different portfolios, which will have their own respective orders.

In [5]:
pf_df = pd.read_csv("data/2/portfolios.csv")
display(pf_df)

portfolio_name portfolio_code currency  \
0  Allocation Service - Example 2 - Portfolio A        PORT001      USD   
1  Allocation Service - Example 2 - Portfolio B        PORT002      USD   

                created_date  
0  2023-04-10T12:00:00+00:00  
1  2023-04-10T12:00:00+00:00

In [6]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "currency",
    },
    "optional": {
        "created": "created_date"
    }
}
result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional={},
        file_type="portfolios",
    )

succ, errors = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "errors": len(errors)}])

success  errors
0        2       0

<h2> Upsert Orders </h2>
Here we have two orders for AMZN stock, both for ten units each with Order 1 being placed 5 minutes before Order 2.

In [7]:
ord_df = pd.read_csv("data/2/orders.csv")
display(ord_df)

portfolio     instrument_name client_internal          isin          figi  \
0   PORT001  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067  BBG000BVPXP1   
1   PORT002  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067  BBG000BVPXP1   

   quantity  price currency order_id side   type state  \
0        10   1550      USD   ORD001  buy  limit   new   
1        10   1550      USD   ORD002  buy  limit   new   

                        date  
0  2023-04-11T11:15:00+00:00  
1  2023-04-11T11:10:00+00:00

In [8]:
set_request = helpers.create_order_set_request_from_df(ord_df, scope)
response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
    order_set_request=set_request)

display(lusid_response_to_data_frame(response))

properties     version.effective_from               version.as_at_date  \
0         {}  0001-01-01 00:00:00+00:00 2023-04-21 15:22:17.408325+00:00   
1         {}  0001-01-01 00:00:00+00:00 2023-04-21 15:22:17.408325+00:00   

             version.as_at_created version.user_id_created  \
0 2023-04-18 14:06:16.656548+00:00    00ucmrzqsbhjXJHmz2p7   
1 2023-04-18 14:06:16.656548+00:00    00ucmrzqsbhjXJHmz2p7   

            version.as_at_modified version.user_id_modified  \
0 2023-04-21 15:22:17.408325+00:00     00ucmrzqsbhjXJHmz2p7   
1 2023-04-21 15:22:17.408325+00:00     00ucmrzqsbhjXJHmz2p7   

   version.as_at_version_number  \
0                            16   
1                            16   

  instrument_identifiers.Instrument/default/Figi  quantity side  \
0                                   BBG000BVPXP1      10.0  buy   
1                                   BBG000BVPXP1      10.0  buy   

           portfolio_id.scope portfolio_id.code                    id.scope  \
0  AllocationService_Example2           PORT002  AllocationService_Example2   
1  AllocationService_Example2           PORT001  AllocationService_Example2   

  id.code instrument_scope lusid_instrument_id state   type  \
0  ORD002          default       LUID_0000MIOX   new  limit   
1  ORD001          default       LUID_0000MIOX   new  limit   

                       date  price.amount price.currency  
0 2023-04-11 11:10:00+00:00          10.0            USD  
1 2023-04-11 11:15:00+00:00          10.0            USD

<h2> Upsert Block </h2>
The Block contains both of our orders.

In [9]:
block_df = pd.read_csv("data/2/blocks.csv")
display(block_df)

block_id      order_ids     instrument_name client_internal          isin  \
0  BLOCK001  ORD001,ORD002  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   

           figi  quantity side   type time_in_force               created_date  
0  BBG000BVPXP1        20  buy  limit           GTC  2023-04-11T11:15:00+00:00

In [10]:
set_request = helpers.create_block_set_request_from_df(block_df, scope)
response = api_factory.build(lusid.api.BlocksApi).upsert_blocks(
    block_set_request=set_request)

display(lusid_response_to_data_frame(response))

id.scope   id.code           order_ids.0.scope  \
0  AllocationService_Example2  BLOCK001  AllocationService_Example2   

  order_ids.0.code           order_ids.1.scope order_ids.1.code properties  \
0           ORD001  AllocationService_Example2           ORD002         {}   

  instrument_identifiers.Instrument/default/Figi lusid_instrument_id  \
0                                   BBG000BVPXP1       LUID_0000MIOX   

   quantity side   type time_in_force              created_date  \
0      20.0  buy  limit           GTC 2023-04-11 11:15:00+00:00   

      version.effective_from               version.as_at_date  \
0  0001-01-01 00:00:00+00:00 2023-04-21 15:22:24.810382+00:00   

             version.as_at_created version.user_id_created  \
0 2023-04-18 14:06:27.380833+00:00    00ucmrzqsbhjXJHmz2p7   

            version.as_at_modified version.user_id_modified  \
0 2023-04-21 15:22:24.810382+00:00     00ucmrzqsbhjXJHmz2p7   

   version.as_at_version_number  
0                            16

<h2> Upsert Placement </h2>
The Placement references our Block.

In [11]:
placement_df = pd.read_csv("data/2/placements.csv")
display(placement_df)

placement_id  block_id     instrument_name client_internal          isin  \
0       PLC001  BLOCK001  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   

           figi  quantity       state side   type time_in_force  \
0  BBG000BVPXP1        20  some_state  buy  limit           GTC   

                created_date  
0  2023-04-11T11:15:00+00:00

In [12]:
set_request = helpers.create_placement_set_request_from_df(placement_df, scope)
response = api_factory.build(lusid.api.PlacementsApi).upsert_placements(
    placement_set_request=set_request)

display(lusid_response_to_data_frame(response))

id.scope id.code           block_ids.0.scope  \
0  AllocationService_Example2  PLC001  AllocationService_Example2   

  block_ids.0.code properties instrument_identifiers.Instrument/default/Figi  \
0         BLOCK001         {}                                   BBG000BVPXP1   

  lusid_instrument_id  quantity       state side time_in_force   type  \
0       LUID_0000MIOX      20.0  some_state  buy           GTC  limit   

               created_date     version.effective_from  \
0 2023-04-11 11:15:00+00:00  0001-01-01 00:00:00+00:00   

                version.as_at_date            version.as_at_created  \
0 2023-04-21 15:22:42.705345+00:00 2023-04-18 14:06:43.429420+00:00   

  version.user_id_created           version.as_at_modified  \
0    00ucmrzqsbhjXJHmz2p7 2023-04-21 15:22:42.705345+00:00   

  version.user_id_modified  version.as_at_version_number  
0     00ucmrzqsbhjXJHmz2p7                            16

<h2> Upsert Executions </h2>
Execution 1 was created a day before Execution 2. They both have the same quantity, but execution 1 has a price of 1560 whereas Execution 2 has a price of 1550.

In [13]:
execution_df = pd.read_csv("data/2/executions.csv")
display(execution_df)

execution_id placement_id     instrument_name client_internal          isin  \
0      EXEC001       PLC001  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   
1      EXEC002       PLC001  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   

           figi  quantity       state side   type  price currency  \
0  BBG000BVPXP1         5  some_state  buy  limit   1560      USD   
1  BBG000BVPXP1         5  some_state  buy  limit   1550      USD   

  settlement_currency  settlement_currency_fx_rate       counterparty  \
0                 USD                          1.0  some_counterparty   
1                 USD                          1.0  some_counterparty   

                created_date  
0  2023-04-11T11:30:00+00:00  
1  2023-04-12T11:30:00+00:00

In [14]:
set_request = helpers.create_execution_set_request_from_df(execution_df, scope)
response = api_factory.build(lusid.api.ExecutionsApi).upsert_executions(
    execution_set_request=set_request)

display(lusid_response_to_data_frame(response))

id.scope  id.code          placement_id.scope  \
0  AllocationService_Example2  EXEC002  AllocationService_Example2   
1  AllocationService_Example2  EXEC001  AllocationService_Example2   

  placement_id.code properties instrument_identifiers.Instrument/default/Figi  \
0            PLC001         {}                                   BBG000BVPXP1   
1            PLC001         {}                                   BBG000BVPXP1   

  lusid_instrument_id  quantity       state side   type  \
0       LUID_0000MIOX       5.0  some_state  buy  limit   
1       LUID_0000MIOX       5.0  some_state  buy  limit   

               created_date  price.amount price.currency settlement_currency  \
0 2023-04-12 11:30:00+00:00        1550.0            USD                 USD   
1 2023-04-11 11:30:00+00:00        1560.0            USD                 USD   

   settlement_currency_fx_rate       counterparty     version.effective_from  \
0                          1.0  some_counterparty  0001-01-01 00:00:00+00:00   
1                          1.0  some_counterparty  0001-01-01 00:00:00+00:00   

                version.as_at_date            version.as_at_created  \
0 2023-04-21 15:22:47.124197+00:00 2023-04-18 14:07:04.628652+00:00   
1 2023-04-21 15:22:47.124197+00:00 2023-04-18 14:07:04.628652+00:00   

  version.user_id_created           version.as_at_modified  \
0    00ucmrzqsbhjXJHmz2p7 2023-04-21 15:22:47.124197+00:00   
1    00ucmrzqsbhjXJHmz2p7 2023-04-21 15:22:47.124197+00:00   

  version.user_id_modified  version.as_at_version_number  
0     00ucmrzqsbhjXJHmz2p7                            16  
1     00ucmrzqsbhjXJHmz2p7                            16

<h2> Upsert Allocations </h2>
These allocation simulate a previous run of the Allocation Service. You can see that Order 1 got alloated 3 shares whereas Order 2 got allocated 2 shares from execution 1 due to it being placed earlier.

In [15]:
allocation_df = pd.read_csv("data/2/allocations.csv")
display(allocation_df)

allocation_id     instrument_name client_internal          isin  \
0      ALLOC001  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   
1      ALLOC002  Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   

           figi  quantity  price currency portfolio_id order_id execution_ids  \
0  BBG000BVPXP1         2   1560      USD      PORT001   ORD001       EXEC002   
1  BBG000BVPXP1         3   1560      USD      PORT002   ORD002       EXEC002   

  placement_ids                       date  
0        PLC001  2023-04-11T11:20:00+00:00  
1        PLC001  2023-04-11T11:20:00+00:00

In [16]:
set_request = helpers.create_allocation_set_request_from_df(allocation_df, scope)
response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
    allocation_set_request=set_request)

display(lusid_response_to_data_frame(response))

id.scope   id.code    allocated_order_id.scope  \
0  AllocationService_Example2  ALLOC002  AllocationService_Example2   
1  AllocationService_Example2  ALLOC001  AllocationService_Example2   

  allocated_order_id.code          portfolio_id.scope portfolio_id.code  \
0                  ORD002  AllocationService_Example2           PORT002   
1                  ORD001  AllocationService_Example2           PORT001   

   quantity instrument_identifiers.Instrument/default/Figi  \
0       3.0                                   BBG000BVPXP1   
1       2.0                                   BBG000BVPXP1   

      version.effective_from               version.as_at_date  \
0  0001-01-01 00:00:00+00:00 2023-04-21 15:22:47.752609+00:00   
1  0001-01-01 00:00:00+00:00 2023-04-21 15:22:47.752609+00:00   

             version.as_at_created version.user_id_created  \
0 2023-04-18 14:13:39.619068+00:00    00ucmrzqsbhjXJHmz2p7   
1 2023-04-18 14:13:39.619068+00:00    00ucmrzqsbhjXJHmz2p7   

            version.as_at_modified version.user_id_modified  \
0 2023-04-21 15:22:47.752609+00:00     00ucmrzqsbhjXJHmz2p7   
1 2023-04-21 15:22:47.752609+00:00     00ucmrzqsbhjXJHmz2p7   

   version.as_at_version_number properties instrument_scope  \
0                            12         {}          default   
1                            12         {}          default   

  lusid_instrument_id       placement_ids.0.scope placement_ids.0.code  \
0       LUID_0000MIOX  AllocationService_Example2               PLC001   
1       LUID_0000MIOX  AllocationService_Example2               PLC001   

          state         side         type                      date  \
0  UnknownState  UnknownSide  UnknownType 2023-04-11 11:20:00+00:00   
1  UnknownState  UnknownSide  UnknownType 2023-04-11 11:20:00+00:00   

   price.amount price.currency       execution_ids.0.scope  \
0        1560.0            USD  AllocationService_Example2   
1        1560.0            USD  AllocationService_Example2   

  execution_ids.0.code  
0              EXEC002  
1              EXEC002

<h2> Upsert Transactions </h2>
You can see that this transaction reflects Allocation 1 being converted to a transaction. This would be a typical EOD process where allocations for this day are converted to transactions.

In [17]:
txn_df = pd.read_csv("data/2/transactions.csv")
display(txn_df)

transaction_id portfolio_id type     instrument_name client_internal  \
0         TXN001      PORT001  Buy  Amazon_Nasdaq_AMZN   imd_346345343   

           isin          figi           transaction_date  \
0  US0231351067  BBG000BVPXP1  2023-04-11T18:00:00+00:00   

             settlement_date  quantity  price  cost currency order_id  \
0  2023-04-13T18:00:00+00:00         2   1560  3120      USD   ORD001   

  allocation_id  
0      ALLOC001

In [18]:
requests = helpers.create_transaction_request_dict_from_df(txn_df, scope)
for portfolio_id in requests:
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
        scope=scope,
        code=portfolio_id,
        transaction_request=requests[portfolio_id]
    )

<h2> Run Allocation Service </h2>

In [19]:
result = api_factory.build(lusid.api.AllocationServiceApi).run_allocation_service([lusid.models.ResourceId(scope,"PLC001")])
display(result)

{'failed': {},
 'values': [{'code': 'PLC001', 'scope': 'AllocationService_Example2'}]}

<h2> Retrieve our Allocations </h2>
You can see that we have 4 allocations, two that we upserted, and another two created by the Allocation Service. This shows how the allocations and execution before the transaction were ignored. Due to the executions being considered separately, you can see that order 1 has two allocations with a quantity of 3. This is because it has received the orphan share for both times. You can also see that the prices have not been averaged.

In [20]:
allocations = api_factory.build(lusid.api.AllocationsApi).list_allocations(filter=f"allocatedOrderId.Scope eq \'{scope}\' and allocatedOrderId.Code in (\'ORD001', \'ORD002\')")
display(lusid_response_to_data_frame(allocations))

id.scope                               id.code  \
0  AllocationService_Example2                              ALLOC002   
1  AllocationService_Example2                              ALLOC001   
2  AllocationService_Example2  2a1121a0-ddd2-46b6-834d-7a899ace4b8f   
3  AllocationService_Example2  ffd73cc1-ca15-48ff-9496-1a117ff9e035   

     allocated_order_id.scope allocated_order_id.code  \
0  AllocationService_Example2                  ORD002   
1  AllocationService_Example2                  ORD001   
2  AllocationService_Example2                  ORD001   
3  AllocationService_Example2                  ORD002   

           portfolio_id.scope portfolio_id.code  quantity  \
0  AllocationService_Example2           PORT002       3.0   
1  AllocationService_Example2           PORT001       2.0   
2  AllocationService_Example2           PORT001       2.0   
3  AllocationService_Example2           PORT002       3.0   

  instrument_identifiers.Instrument/default/Figi     version.effective_from  \
0                                   BBG000BVPXP1  0001-01-01 00:00:00+00:00   
1                                   BBG000BVPXP1  0001-01-01 00:00:00+00:00   
2                                            NaN  0001-01-01 00:00:00+00:00   
3                                            NaN  0001-01-01 00:00:00+00:00   

                version.as_at_date properties instrument_scope  \
0 2023-04-21 15:22:47.752609+00:00         {}          default   
1 2023-04-21 15:22:47.752609+00:00         {}          default   
2 2023-04-21 15:22:51.872170+00:00         {}          default   
3 2023-04-21 15:22:51.872170+00:00         {}          default   

  lusid_instrument_id       placement_ids.0.scope placement_ids.0.code  \
0       LUID_0000MIOX  AllocationService_Example2               PLC001   
1       LUID_0000MIOX  AllocationService_Example2               PLC001   
2       LUID_0000MIOX  AllocationService_Example2               PLC001   
3       LUID_0000MIOX  AllocationService_Example2               PLC001   

                state         side         type  \
0        UnknownState  UnknownSide  UnknownType   
1        UnknownState  UnknownSide  UnknownType   
2  PartiallyAllocated          buy        limit   
3  PartiallyAllocated          buy        limit   

                              date  price.amount price.currency  \
0        2023-04-11 11:20:00+00:00        1560.0            USD   
1        2023-04-11 11:20:00+00:00        1560.0            USD   
2 2023-04-21 15:22:51.699596+00:00        1560.0            USD   
3 2023-04-21 15:22:51.699382+00:00        1560.0            USD   

  settlement_currency       execution_ids.0.scope execution_ids.0.code  \
0                None  AllocationService_Example2              EXEC002   
1                None  AllocationService_Example2              EXEC002   
2                 USD  AllocationService_Example2              EXEC001   
3                 USD  AllocationService_Example2              EXEC001   

  instrument_identifiers.Instrument/default/LusidInstrumentId  
0                                                NaN           
1                                                NaN           
2                                      LUID_0000MIOX           
3                                      LUID_0000MIOX